In [2]:
!pip install xgboost

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import sklearn
from xgboost import XGBClassifier
import xgboost
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pickle
from sklearn.utils import shuffle

In [26]:
df = pd.read_csv("C:/Users/janine van wonderen/Desktop/dmt2/training_set_VU_DM.csv",delimiter=",", 
                 usecols = ['srch_id', 'site_id', 'visitor_location_country_id','visitor_hist_starrating', 
                            'visitor_hist_adr_usd', 'prop_country_id','prop_id', 'prop_starrating', 
                            'prop_review_score', 'prop_brand_bool','prop_location_score1', 'prop_location_score2',
                            'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
                            'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window','srch_adults_count', 
                            'srch_children_count', 'srch_room_count','srch_saturday_night_bool', 'srch_query_affinity_score',
                            'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'click_bool', 
                            'gross_bookings_usd','booking_bool'])
df = shuffle(df)

Lets train it on clicking

In [27]:
trainOnlyColumns = ['position','click_bool','booking_bool','gross_bookings_usd']
Y = df['click_bool']
df.drop(trainOnlyColumns, axis=1,inplace=True)

In [11]:
n = 50
numOfRowsTest = (int(len(df)*(n/100)))
X_test = df.tail(numOfRowsTest)
Y_test = Y[-numOfRowsTest:]
df.drop(df.tail(numOfRowsTest).index,inplace=True)
y_train = Y[:(len(Y)-numOfRowsTest)]

In [12]:
# construct xgboost.DMatrix from numpy array, treat -999.0 as missing value
xgmat = xgboost.DMatrix( df, label=y_train)

# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'reg:logistic'
# scale weight of positive examples
param['scale_pos_weight'] = sum(y_train==0)/sum(y_train==1)
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['silent'] = 0
param['nthread'] = 10

# you can directly throw param in, though we want to watch multiple metrics here
plst = list(param.items())+[('eval_metric', 'ams@0.15')]

watchlist = [ (xgmat,'train') ]
# boost 120 trees
num_round = 120
print ('loading data end, start to boost trees')
bst = xgboost.train( plst, xgmat, num_round, watchlist );
# save out model
bst.save_model('paraTest.model')

print ('finish training')

# fit model no training data
#model = XGBClassifier()
#model.fit(xs, ys)
# make predictions for test data

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


loading data end, start to boost trees
[0]	train-auc:0.664911	train-ams@0.15:63.0607
[1]	train-auc:0.668627	train-ams@0.15:63.8916
[2]	train-auc:0.670858	train-ams@0.15:64.5629
[3]	train-auc:0.672682	train-ams@0.15:64.9084
[4]	train-auc:0.673955	train-ams@0.15:65.5787
[5]	train-auc:0.674724	train-ams@0.15:65.5469
[6]	train-auc:0.675498	train-ams@0.15:65.8522
[7]	train-auc:0.676444	train-ams@0.15:65.9387
[8]	train-auc:0.67717	train-ams@0.15:66.2636
[9]	train-auc:0.678222	train-ams@0.15:66.4067
[10]	train-auc:0.678887	train-ams@0.15:66.4968
[11]	train-auc:0.679659	train-ams@0.15:66.7388
[12]	train-auc:0.680349	train-ams@0.15:66.8748
[13]	train-auc:0.680975	train-ams@0.15:67.0357
[14]	train-auc:0.681515	train-ams@0.15:67.1912
[15]	train-auc:0.68201	train-ams@0.15:67.2054
[16]	train-auc:0.682724	train-ams@0.15:67.2124
[17]	train-auc:0.683505	train-ams@0.15:67.4246
[18]	train-auc:0.684065	train-ams@0.15:67.4971
[19]	train-auc:0.684587	train-ams@0.15:67.536
[20]	train-auc:0.685103	train-ams@

In [13]:
xgmat_test = xgboost.DMatrix( X_test, label=Y_test)
y_pred = bst.predict(xgmat_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
Counter(predictions)

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Accuracy: 67.03%


Counter({0.0: 1638283, 1.0: 840890})

In [14]:
df.columns

Index(['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv'],
      dtype='object')

In [38]:
df2 = df[['site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv']]

In [39]:
n = 50
numOfRowsTest = (int(len(df2)*(n/100)))
X_test = df2.tail(numOfRowsTest)
Y_test = Y[-numOfRowsTest:]
df2.drop(df2.tail(numOfRowsTest).index,inplace=True)
y_train = Y[:(len(Y)-numOfRowsTest)]

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [46]:
# construct xgboost.DMatrix from numpy array, treat -999.0 as missing value
xgmat = xgboost.DMatrix( df2, label=y_train)

# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'reg:logistic'
# scale weight of positive examples
param['scale_pos_weight'] = sum(y_train==0)/sum(y_train==1)
param['eta'] = 0.1
param['max_depth'] = 8
param['eval_metric'] = 'auc'
param['silent'] = 0
param['nthread'] = 10
param['tree_method'] = 'hist'

# you can directly throw param in, though we want to watch multiple metrics here
plst = list(param.items())+[('eval_metric', 'ams@0.15')]

watchlist = [ (xgmat,'train') ]
# boost 120 trees
num_round = 120
print ('loading data end, start to boost trees')
bst = xgboost.train( plst, xgmat, num_round, watchlist );
# save out model
bst.save_model('paraTest2.model')

print ('finish training')

# fit model no training data
#model = XGBClassifier()
#model.fit(xs, ys)
# make predictions for test data

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


loading data end, start to boost trees
[0]	train-auc:0.673874	train-ams@0.15:64.718
[1]	train-auc:0.678764	train-ams@0.15:65.8964
[2]	train-auc:0.680713	train-ams@0.15:66.1788
[3]	train-auc:0.681925	train-ams@0.15:66.6911
[4]	train-auc:0.682658	train-ams@0.15:67.0074
[5]	train-auc:0.68387	train-ams@0.15:67.2284
[6]	train-auc:0.684875	train-ams@0.15:67.391
[7]	train-auc:0.68581	train-ams@0.15:67.6793
[8]	train-auc:0.686907	train-ams@0.15:67.8509
[9]	train-auc:0.687972	train-ams@0.15:68.1499
[10]	train-auc:0.688869	train-ams@0.15:68.1889
[11]	train-auc:0.689983	train-ams@0.15:68.6084
[12]	train-auc:0.691152	train-ams@0.15:68.759
[13]	train-auc:0.691995	train-ams@0.15:68.8741
[14]	train-auc:0.692953	train-ams@0.15:69.1008
[15]	train-auc:0.69392	train-ams@0.15:69.4765
[16]	train-auc:0.694828	train-ams@0.15:69.7176
[17]	train-auc:0.695859	train-ams@0.15:69.9428
[18]	train-auc:0.696763	train-ams@0.15:70.012
[19]	train-auc:0.697607	train-ams@0.15:70.1592
[20]	train-auc:0.698403	train-ams@0.15

In [47]:
xgmat_test = xgboost.DMatrix( X_test, label=Y_test)
y_pred = bst.predict(xgmat_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
Counter(predictions)

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Accuracy: 68.87%


Counter({1.0: 791162, 0.0: 1688011})

In [34]:
df3 = df[["prop_location_score2", "srch_query_affinity_score", "promotion_flag","prop_starrating", 
         "random_bool", "srch_length_of_stay", "prop_review_score"]]

In [35]:
n = 50
numOfRowsTest = (int(len(df3)*(n/100)))
X_test = df3.tail(numOfRowsTest)
Y_test = Y[-numOfRowsTest:]
df3.drop(df3.tail(numOfRowsTest).index,inplace=True)
y_train = Y[:(len(Y)-numOfRowsTest)]

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [36]:
# construct xgboost.DMatrix from numpy array, treat -999.0 as missing value
xgmat = xgboost.DMatrix( df3, label=y_train)

# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'reg:logistic'
# scale weight of positive examples
param['scale_pos_weight'] = sum(y_train==0)/sum(y_train==1)
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['silent'] = 0
param['nthread'] = 10

# you can directly throw param in, though we want to watch multiple metrics here
plst = list(param.items())+[('eval_metric', 'ams@0.15')]

watchlist = [ (xgmat,'train') ]
# boost 120 trees
num_round = 120
print ('loading data end, start to boost trees')
bst = xgboost.train( plst, xgmat, num_round, watchlist );
# save out model
bst.save_model('paraTest.model')

print ('finish training')

# fit model no training data
#model = XGBClassifier()
#model.fit(xs, ys)
# make predictions for test data

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


loading data end, start to boost trees
[0]	train-auc:0.641337	train-ams@0.15:50.3717
[1]	train-auc:0.642473	train-ams@0.15:50.6822
[2]	train-auc:0.642932	train-ams@0.15:50.592
[3]	train-auc:0.643611	train-ams@0.15:51.3104
[4]	train-auc:0.643857	train-ams@0.15:51.4545
[5]	train-auc:0.644436	train-ams@0.15:51.6317
[6]	train-auc:0.644883	train-ams@0.15:52.2884
[7]	train-auc:0.645283	train-ams@0.15:52.3058
[8]	train-auc:0.645566	train-ams@0.15:52.4049
[9]	train-auc:0.645787	train-ams@0.15:52.4153
[10]	train-auc:0.646281	train-ams@0.15:52.4988
[11]	train-auc:0.646555	train-ams@0.15:52.5544
[12]	train-auc:0.646763	train-ams@0.15:52.6431
[13]	train-auc:0.647018	train-ams@0.15:52.6379
[14]	train-auc:0.647387	train-ams@0.15:52.7735
[15]	train-auc:0.647625	train-ams@0.15:52.8535
[16]	train-auc:0.647765	train-ams@0.15:52.8709
[17]	train-auc:0.647993	train-ams@0.15:52.9596
[18]	train-auc:0.648217	train-ams@0.15:53.0883
[19]	train-auc:0.648377	train-ams@0.15:53.0605
[20]	train-auc:0.64852	train-ams

In [37]:
xgmat_test = xgboost.DMatrix( X_test, label=Y_test)
y_pred = bst.predict(xgmat_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
Counter(predictions)

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\janine van wonderen\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Accuracy: 57.30%


Counter({1.0: 1092187, 0.0: 1386986})

Lets train it on booking

In [8]:
trainOnlyColumns = ['position','click_bool','booking_bool','gross_bookings_usd']
Y = df['booking_bool']
df.drop(trainOnlyColumns, axis=1,inplace=True)

In [9]:
n = 20
numOfRowsTest = (int(len(df)*(n/100)))
X_test = df.tail(numOfRowsTest)
Y_test = Y[-numOfRowsTest:]
df.drop(df.tail(numOfRowsTest).index,inplace=True)
y_train = Y[:(len(Y)-numOfRowsTest)]

In [ ]:
# construct xgboost.DMatrix from numpy array, treat -999.0 as missing value
xgmat = xgboost.DMatrix( df, label=y_train)

# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'reg:logistic'
# scale weight of positive examples
param['scale_pos_weight'] = sum(y_train==0)/sum(y_train==1)
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['silent'] = 0
param['nthread'] = 10

# you can directly throw param in, though we want to watch multiple metrics here
plst = list(param.items())+[('eval_metric', 'ams@0.15')]

watchlist = [ (xgmat,'train') ]
# boost 120 trees
num_round = 25
print ('loading data end, start to boost trees')
bst = xgboost.train( plst, xgmat, num_round, watchlist , early_stopping_rounds=10);
# save out model
bst.save_model('bookingModel.model')

print ('finish training')

# fit model no training data
#model = XGBClassifier()
#model.fit(xs, ys)
# make predictions for test data

In [ ]:
xgmat_test = xgboost.DMatrix( X_test, label=Y_test)
y_pred = bst.predict(xgmat_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
Counter(predictions)

Lets make a submission

In [7]:
bst = xgboost.Booster({'nthread': 12})  # init model
bst.load_model('xgboostBinary26April.model')  # load data


In [ ]:
dfTest = pd.read_csv('../data/test.csv', delimiter=",", usecols = ['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price',  'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv'])
xgmat_test_submission = xgboost.DMatrix( dfTest)

In [ ]:
y_pred_clicking = bst.predict(xgmat_test_submission)


In [ ]:
bst = xgboost.Booster({'nthread': 12})  # init model
bst.load_model('bookingModel.model')  # load data

In [ ]:
y_pred_booking = bst.predict(xgmat_test_submission)

In [ ]:
y_pred = y_pred_clicking + y_pred_booking
dfTest['predictedPos'] = y_pred
dfTest = dfTest.sort_values(['srch_id','predictedPos'],ascending=[True, False])


In [ ]:
print(dfTest.head())
dfSubmission = dfTest[['srch_id','prop_id']]
dfSubmission.to_csv("submissionTest2.csv",index=False)

In [ ]:
#Counter(y_pred)
predictions = [round(value) for value in y_pred]
Counter(predictions)